In [242]:
from dotenv import load_dotenv
import os
import warnings

load_dotenv()
warnings.filterwarnings('ignore')

# ChatBot

In [190]:
from sentence_transformers import SentenceTransformer
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
# from langchain.prompts import ChatPromptTemplate
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_mistralai import ChatMistralAI
from typing import Literal
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_core.runnables import RunnableLambda
from langchain.chains import RetrievalQA

In [7]:
sentence = ['''ਪੰਜਾਬ ਦੀਆਂ ਖੇਤੀ-ਭੂਗੋਲਿਕ ਹਾਲਾਤਾਂ ਨਿੰਬੂ ਜਾਤੀ ਦੇ ਫ਼ਲਾਂ ਦੇ ਉਤਪਾਦਨ ਲਈ ਸਭ ਤੋਂ
ਵੱਧ ਢੁੱਕਵੀਆਂ ਹਨ। ਦੂਜੇ ਫ਼ਲਾਂ ਦੇ ਮੁਕਾਬਲੇ ਆਪਣੀ ਉੱਚ ਆਰਥਿਕ ਉਤਪਾਦਕਤਾ ਕਾਰਨ
ਨਿੰਬੂ ਜਾਤੀ ਦੇ ਫ਼ਲਾਂ ਦੀ ਖੇਤੀ ਕਿਸਾਨਾਂ ਲਈ ਵਰਦਾਨ ਸਿੱਧ ਹੋਈ ਹੈ। ਇਹ ਫ਼ਲ ਰਕਬਾ ਅਤੇ
ਉਤਪਾਦਨ ਦੋਨਾਂ ਵਿੱਚ ਨੰਬਰ ਇੱਕ ਦੇ ਤੌਰ ਤੇ ਉਭਰੇ ਹਨ ਅਤੇ ਪੰਜਾਬ ਰਾਜ ਵਿਚ “ਸੁਨਹਿਰੀ
ਕ੍ਰਾਂਤੀ” ਲਿਆਏ ਹਨ। ਪਰ ਫੁੱਲ ਪੈਣ ਤੋਂ ਫ਼ਲ ਦੇ ਪੱਕਣ ਸਮੇਂ ਤੱਕ ਫ਼ੁੱਲਾਂ ਅਤੇ ਫ਼ਲਾਂ ਦਾ ਭਾਰੀ
ਮਾਤਰਾ ਵਿੱਚ ਝੜ ਜਾਣਾ ਨਿੰਬੂ ਦੇ ਫ਼ਲਾਂ ਵਿਚ ਇੱਕ ਗੰਭੀਰ ਸਮੱਸਿਆ ਹੈ।''']
# embeddings = model.encode(sentence)

In [30]:
# model_name = 'l3cube-pune/punjabi-sentence-bert-nli'
model_name = 'l3cube-pune/punjabi-sentence-similarity-sbert'
embeddings = HuggingFaceEmbeddings(
    model_name=model_name  # Provide the pre-trained model's path
    # model_kwargs=model_kwargs, # Pass the model configuration options
    # encode_kwargs=encode_kwargs # Pass the encoding options
)

In [22]:
def get_doc(path):
    f = open(path)
    sample_doc = f.read()
    f.close()
    sample_doc = sample_doc.replace('\n\n','#sentencesahikarnekeliyerandomvariable')
    sample_doc = sample_doc.replace('\n' ,'')
    sample_doc = sample_doc.replace('#sentencesahikarnekeliyerandomvariable','\n\n')
    return sample_doc

### Creating vector stores

In [62]:
vectorstore = None
vectorstore = Chroma(persist_directory=persist_directory, embedding_function=embeddings)
retriever = vectorstore.as_retriever(search_kwargs={'k':1})

In [227]:
# prompt = ChatPromptTemplate.from_messages(
#     [
#         (
#             "system",
#             """You are an employ at call center, provide answer to the question from your own knowlage, you may use the provided context
#             context: {context}
#             """,
#         ),
#         ("human", "{question}"),
#     ]
# )

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """ਤੁਸੀਂ ਕਾਲ ਸੈਂਟਰ ਵਿੱਚ ਇੱਕ ਕਰਮਚਾਰੀ ਹੋ, ਆਪਣੇ ਖੁਦ ਦੇ ਗਿਆਨ ਤੋਂ ਸਵਾਲ ਦਾ ਜਵਾਬ ਪ੍ਰਦਾਨ ਕਰੋ, ਤੁਸੀਂ ਪ੍ਰਦਾਨ ਕੀਤੇ ਸੰਦਰਭ ਦੀ ਵਰਤੋਂ ਕਰ ਸਕਦੇ ਹੋ
            ਸੰਦਰਭ: {context}
            """,
        ),
        ("human", "{question}"),
    ]
)

In [228]:
llm = ChatMistralAI(
    model="mistral-large-latest"
)

In [237]:
rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [158]:
sample_chain = ({"context": fruit_retriever, "question": RunnablePassthrough()} | prompt )
sample_chain.invoke(question)

ChatPromptValue(messages=[SystemMessage(content='ਤੁਸੀਂ ਕਾਲ ਸੈਂਟਰ ਵਿੱਚ ਇੱਕ ਕਰਮਚਾਰੀ ਹੋ, ਆਪਣੇ ਖੁਦ ਦੇ ਗਿਆਨ ਤੋਂ ਸਵਾਲ ਦਾ ਜਵਾਬ ਪ੍ਰਦਾਨ ਕਰੋ, ਤੁਸੀਂ ਪ੍ਰਦਾਨ ਕੀਤੇ ਸੰਦਰਭ ਦੀ ਵਰਤੋਂ ਕਰ ਸਕਦੇ ਹੋ\n            ਸੰਦਰਭ: [Document(page_content="page - 554. ਅੰਬ\\n\\nਅੰਬ ਭਾਰਤ ਦੇ ਲੋਕਾਂ ਦਾ ਮਨਪਸੰਦ ਫਲ ਹੈ । ਇਸ ਵਿੱਚ ਸਾਰੇ ਫਲਾਂ ਤੋਂਜ਼ਿਆਦਾ ਵਿਟਾਮਿਨ \'ਏ” (4800 ਯੂਨਿਟ) ਅਤੇ ਵਿਟਾਮਿਨ ਬੀ-1, ਵਿਟਾਮਿਨਬੀ-2 90 ਮਿਲੀਗ੍ਰਾਮ, ਵਿਟਾਮਿਨ ਸੀ 13 ਮਿਲੀਗ੍ਰਾਮ ਪੁਤੀ 100 ਗ੍ਰਾਮ ਗੁੱਦੇਦੇ ਹਿਸਾਬ ਨਾਲ ਹੁੰਦੇ ਹਨ । ਇਸ ਤੋਂ ਇਲਾਵਾ ਮਿਠਾਸ (11.8%) ਤੇ ਕਈਮਾਤਰਾ ਵਿੱਚ ਪਾਈਆਂ ਜਾਦੀਆਂ ਹਨ । ਇਹ ਫਲ ਵਾਧੇ ਦੀ ਹਰ ਅਵੱਸਥਾ ਤੇਵਰਤਿਆ ਜਾ ਸਕਦਾ ਹੈ ।ਪੌਣ ਪਾਣੀ ਤੇ ਜ਼ਮੀਨ\\n\\nਅੰਬ ਕਈ ਤਰ੍ਹਾਂ ਦੀਆਂ ਜ਼ਮੀਨਾਂ ਵਿੱਚ ਉਗਾਇਆ ਜਾ ਸਕਦਾ ਹੈ ਪਰਇਸ ਲਈ ਜ਼ਮੀਨ ਡੂੰਘੀ ਹੋਵੇ ਜਿਸ ਵਿੱਚ ਸਖਤ ਰੋੜ ਆਦਿ ਦੀ ਤਹਿ 6 ਫੁੱਟਤੱਕ ਨਹੀਂ ਹੋਣੀ ਚਾਹੀਦੀ । ਜ਼ਮੀਨ ਵਿੱਚ ਬਹੁਤਾ ਨਮਕੀਨ ਪਦਾਰਥ ਨਾ ਹੋਵੇਅਤੇ ਬਿਜਲੀ ਦੀ ਚਾਲਕਤਾ ਦੀ ਹੱਦ 10 ਮਿਲੀ ਮਹੋਜ਼/ਸੈਂਟੀਮੀਟਰ ਤੋਂ ਜ਼ਿਆਦਾਨਾ ਹੋਵੇ । ਕੈਲਸ਼ੀਅਮ ਕਾਰਬੋਨੇਟ ਅਤੇ ਚੂਨਾ ਕੁਮਵਾਰ 10% ਅਤੇ 20% ਤੋਂਘੋਟ ਹੋਣੇ ਚਾਹੀਦੇ ਹਨ । ਜ਼ਮੀਨ ਦੀ ਰਸਾਇਣਕ ਕਿਰਿਆ (ਪੀ ਐਚ) 60ਸੈਂਟੀਮੀਟਰ ਤੱਕ 8.5 ਤੋਂ ਜ਼ਿਆਦਾ ਨਾ ਹੋਵੇ। ਪਾਣੀ ਦੀ ਸਤਹਿ ਧਰਤੀ ਵਿੱਚ 3ਮੀਟਰ ਤੋਂ ਹੇਠਾਂ ਹੋਵੇ ਤਾਂ ਚੰਗ

In [244]:
question = 'ਅੰਬ ਦੇ ਰੁੱਖ ਲਈ ਮਿੱਟੀ ਦੀ ਆਦਰਸ਼ ਸਥਿਤੀ ਕੀ ਹੋਣੀ ਚਾਹੀਦੀ ਹੈ'
print(rag_chain.invoke(question))

ਅੰਬ ਦੇ ਰੁੱਖ ਲਈ ਮਿੱਟੀ ਦੀ ਆਦਰਸ਼ ਸਥਿਤੀ ਹੇਠ ਲਿਖੀਆਂ ਸ਼ਰਤਾਂ ਨਾਲ ਹੋਣੀ ਚਾਹੀਦੀ ਹੈ:

1. **ਜ਼ਮੀਨ ਡੂੰਘੀ ਹੋਵੇ**: ਜਿਸ ਵਿੱਚ ਸਖਤ ਰੋੜ ਆਦਿ ਦੀ ਤਹਿ 6 ਫੁੱਟ ਤੱਕ ਨਹੀਂ ਹੋਣੀ ਚਾਹੀਦੀ।
2. **ਜ਼ਮੀਨ ਵਿੱਚ ਬਹੁਤਾ ਨਮਕੀਨ ਪਦਾਰਥ ਨਾ ਹੋਵੇ**: ਬਿਜਲੀ ਦੀ ਚਾਲਕਤਾ ਦੀ ਹੱਦ 10 ਮਿਲੀ ਮਹੋਜ਼/ਸੈਂਟੀਮੀਟਰ ਤੋਂ ਜ਼ਿਆਦਾ ਨਾ ਹੋਵੇ।
3. **ਕੈਲਸ਼ੀਅਮ ਕਾਰਬੋਨੇਟ ਅਤੇ ਚੂਨਾ**: 10% ਅਤੇ 20% ਤੋਂ ਘੋਟ ਹੋਣੇ ਚਾਹੀਦੇ ਹਨ।
4. **ਜ਼ਮੀਨ ਦੀ ਰਸਾਇਣਕ ਕਿਰਿਆ (ਪੀ ਐਚ)**: 6.0 ਸੈਂਟੀਮੀਟਰ ਤੱਕ 8.5 ਤੋਂ ਜ਼ਿਆਦਾ ਨਾ ਹੋਵੇ।
5. **ਪਾਣੀ ਦੀ ਸਤਹਿ**: ਧਰਤੀ ਵਿੱਚ 3 ਮੀਟਰ ਤੋਂ ਹੇਠਾਂ ਹੋਵੇ ਤਾਂ ਚੰਗੀ ਗੱਲ ਹੈ।

ਇਹ ਸਾਰੀਆਂ ਸ਼ਰਤਾਂ ਪੂਰੀਆਂ ਹੋਣ 'ਤੇ ਅੰਬ ਦੇ ਰੁੱਖ ਦਾ ਵਾਧਾ ਸਹੀ ਢੰਗ ਨਾਲ ਹੋ ਸਕਦਾ ਹੈ।


# Image input

In [16]:
import cv2
import pytesseract as pt

In [28]:
def get_text(image_path):
    img = cv2.imread(image_path)
    text = pt.image_to_string(img,config='--psm 6',lang='pan')
    return text

In [22]:
path = 'ss_test.png'
get_text(path)

'ਫਲਦਾਰ ਪੋਦਿਆਂ ਲਈ ਕਿਹੜੀ ਜ਼ਮੀਨ ਸਭ ਤੋਂ ਵਧੀਆ ਹੈ\n\x0c'

In [33]:
question = get_text('ss_test.png')
rag_chain.invoke(question)

'ਫਲਦਾਰ ਪੌਦਿਆਂ ਲਈ ਸਭ ਤੋਂ ਵਧੀਆ ਜ਼ਮੀਨ ਉਹ ਹੁੰਦੀ ਹੈ ਜੋ ਚੰਗੇ ਜਲਨਿਕਾਸ ਦੀ ਸਮਰੱਥਾ ਰੱਖਦੀ ਹੈ, ਡੂੰਘੀ ਅਤੇ 120 ਸੈਂਟੀਮੀਟਰ ਦੀ ਡੂੰਘਾਈ ਤੱਕ ਕੰਕਰ-ਪੇਥਰ ਦੀ ਤਹਿ ਤੋਂ ਰਹਿਤ ਹੋਵੇ। ਮਿੱਟੀ ਦੇ ਭੌਤਿਕ ਗੁਣਾਂ ਵਿੱਚ ਬਿਜਲੀ ਦੀ ਚਾਲਕਤਾ 0.5 ਮਿਲੀ ਮਹੋਜ਼ ਪੁਤੀ ਸੈਂਟੀਮੀਟਰ ਤੋਂ ਹੇਠ, ਕੈਲਸ਼ੀਅਮ ਕਾਰਬੋਨੇਟ 5% ਤੋਂ ਘੱਟ, ਚੂਨਾ 10% ਤੋਂ ਘੱਟ ਅਤੇ ਪੀਐਚ 6 ਤੋਂ 8 ਤੱਕ ਹੋਣਾ ਚਾਹੀਦਾ ਹੈ। ਭਾਰੀ, ਕੌਲਰ ਅਤੇ ਸੇਮ ਵਾਲੀ ਜ਼ਮੀਨ ਫਲਦਾਰ ਪੌਦਿਆਂ ਲਈ ਢੁੱਕਵੀ ਨਹੀਂ ਹੁੰਦੀ।'

# Routing

In [245]:
def get_retriever(persist_directory):
    vectorstore = Chroma(persist_directory=persist_directory, embedding_function=embeddings)
    retriever = vectorstore.as_retriever(search_kwargs={'k':1})
    return retriever

fruit_retriever = get_retriever('vectorstore/fruit/')
vegetable_retriever = get_retriever('vectorstore/vegetable/')
rabi_retriever = get_retriever('vectorstore/rabi/')
kharif_retriever = get_retriever('vectorstore/kharif/')

In [246]:
class RouteQuery(BaseModel):
    """Route a user query to the most relevant datasource."""

    datasource: Literal["fruits", "vegetable", "rabi_crop","kharif_crop"] = Field(
        ...,
        description="Given a user question choose which datasource would be most relevant for answering their question",
    )

In [247]:
system = """You are an expert at routing a user question to the appropriate data source.

Based on the what whether the question is about fruits, vegetable or rabi or kharif crop; route it to the relevant data source."""

prompt_routing = ChatPromptTemplate.from_messages(
    [
        ("system",system),
        ("human", "{question}"),
    ]
)

In [265]:
# question = """what is the ideal soil condition for mango plants"""
# question = """what is the ideal soil condition for wheat"""
question = "ਕਣਕ ਲਈ ਮਿੱਟੀ ਦੀ ਆਦਰਸ਼ ਸਥਿਤੀ ਕੀ ਹੈ"
# question = "ਅੰਬ ਦੇ ਪੌਦਿਆਂ ਲਈ ਮਿੱਟੀ ਦੀ ਆਦਰਸ਼ ਸਥਿਤੀ ਕੀ ਹੈ"
# question = 'ਸੰਤਰੇ ਦੇ ਪੌਦਿਆਂ ਲਈ ਮਿੱਟੀ ਦੀ ਆਦਰਸ਼ ਸਥਿਤੀ ਕੀ ਹੈ'

In [268]:
def router(query):
    structured_llm = llm.with_structured_output(RouteQuery)
    router = prompt_routing | structured_llm
    result = router.invoke({"question": query})
    return result.datasource.lower()

In [269]:
def create_rag_chain(retriever):
    rag_chain = (
        {"context": retriever, "question": RunnablePassthrough()}
        | prompt
        | llm
        | StrOutputParser()
    )
    return rag_chain

In [274]:
import time
def route(query):
    domain = router(query)
    if "fruits" in domain:
        rag_chain = create_rag_chain(fruit_retriever)
    elif "vegetable" in domain:
        rag_chain = create_rag_chain(vegetable_retriever)
    elif "rabi" in domain:
        rag_chain = create_rag_chain(rabi_retriever)
    else:
        rag_chain = create_rag_chain(kharif_retriever)
    print(domain)
    time.sleep(1)
    print('-'*20)
    return rag_chain.invoke(query)
    # return rag_chain

In [275]:
result = route(question)
print(result)

rabi_crop
--------------------
ਕਣਕ ਲਈ ਮਿੱਟੀ ਦੀ ਆਦਰਸ਼ ਸਥਿਤੀ ਦਰਮਿਆਨੀ ਤੋਂ ਭਾਰੀ ਜ਼ਮੀਨ ਹੈ। ਇਹ ਜ਼ਮੀਨ ਕੱਲਰ ਅਤੇ ਸੇਮ ਵਾਲੀਆਂ ਜ਼ਮੀਨਾਂ ਤੋਂ ਬਿਨਾਂ ਹਰ ਕਿਸਮ ਦੀ ਜ਼ਮੀਨ ਹੋ ਸਕਦੀ ਹੈ। ਕਣਕ ਦੀ ਬਿਜਾਈ ਲਈ ਸਹੀ ਸਮੇਂ ਤੇ ਬਿਜਾਈ ਕਰਨਾ ਜ਼ਰੂਰੀ ਹੈ, ਅਤੇ ਉਹਨਾਂ ਜ਼ਮੀਨਾਂ ਵਿੱਚ ਜਿਹਨਾਂ ਵਿੱਚ ਕੱਲਰ ਅਤੇ ਸੇਮ ਦੀ ਮਾਤਰਾ ਜ਼ਿਆਦਾ ਨਹੀਂ ਹੈ, ਕਣਕ ਦੀ ਬਿਜਾਈ ਮਿਲਦੀ ਹੈ।
